# Cumulative

- Every process updates the statistics just before going `idle`, so it's not instant and `in progress` queries do not effect it (including the current transaction)
- Statistics are cached per transaction making them static through a single transaction (by default, can be modified `stats_fetch_consistency`)
- An exception to both ot the above rules are `pg_stat_xact*` views that show current transaction statistics and are changing dynamically through the transaction (basically using the `local` statistics and the ones stored in `shared memory`)

## What To Monitor

- Access / Usage of objects
- I/O - disk and buffer reads and writes
- Replication / WAL
- Archiving

## How To Monitor

The amount of provided cumulative statistics views is a bit overwhelming, the important part is to understand which data can be found here by category. Then, one can understand what to look for and where.

Category                |    View Name                  |   Description
--------                |    -------                    |   --------
General                 |   pg_stat_database	        |   One row per database, showing database-wide statistics.
Archiving               |   pg_stat_archiver	        |   One row only, showing statistics about the WAL archiver process's activity.
Replication             |   pg_stat_database_conflicts	|   One row per database, showing database-wide statistics about query cancels due to conflict with recovery on standby servers.
Replication             |   pg_stat_replication_slots	|   One row per replication slot, showing statistics about the replication slot's usage.
Replication             |   pg_stat_subscription_stats	|   One row per subscription, showing statistics about errors.
Replication             |   pg_stat_wal	                |   One row only, showing statistics about WAL activity.
Access                  |   pg_stat_all_tables	        |   One row for each table in the current database, showing statistics about accesses to that specific table.
Access                  |   pg_stat_sys_tables	        |   Same as pg_stat_all_tables, except that only system tables are shown.
Access                  |   pg_stat_user_tables	        |   Same as pg_stat_all_tables, except that only user tables are shown.
Access                  |   pg_stat_xact_all_tables	    |   Similar to pg_stat_all_tables, but counts actions taken so far within the current transaction (which are not yet included in pg_stat_all_tables and related views). The columns for numbers of live and dead rows and vacuum and analyze actions are not present in this view.
Access                  |   pg_stat_xact_sys_tables	    |   Same as pg_stat_xact_all_tables, except that only system tables are shown.
Access                  |   pg_stat_xact_user_tables	|   Same as pg_stat_xact_all_tables, except that only user tables are shown.
Access                  |   pg_stat_all_indexes	        |   One row for each index in the current database, showing statistics about accesses to that specific index.
Access                  |   pg_stat_sys_indexes	        |   Same as pg_stat_all_indexes, except that only indexes on system tables are shown.
Access                  |   pg_stat_user_indexes	    |   Same as pg_stat_all_indexes, except that only indexes on user tables are shown.
Access                  |   pg_stat_user_functions	    |   One row for each tracked function, showing statistics about executions of that function.
Access                  |   pg_stat_xact_user_functions |   Similar to pg_stat_user_functions, but counts only calls during the current transaction (which are not yet included in pg_stat_user_functions).
I/O                     |   pg_stat_bgwriter	        |   One row only, showing statistics about the background writer process's activity.
I/O                     |   pg_stat_io	                |   One row for each combination of backend type, context, and target object containing cluster-wide I/O statistics.
I/O                     |   pg_stat_slru                |   One row per SLRU, showing statistics of operations.
I/O                     |   pg_statio_all_tables	    |   One row for each table in the current database, showing statistics about I/O on that specific table.
I/O                     |   pg_statio_sys_tables	    |   Same as pg_statio_all_tables, except that only system tables are shown.
I/O                     |   pg_statio_user_tables	    |   Same as pg_statio_all_tables, except that only user tables are shown.
I/O                     |   pg_statio_all_indexes	    |   One row for each index in the current database, showing statistics about I/O on that specific index.
I/O                     |   pg_statio_sys_indexes	    |   Same as pg_statio_all_indexes, except that only indexes on system tables are shown.
I/O                     |   pg_statio_user_indexes	    |   Same as pg_statio_all_indexes, except that only indexes on user tables are shown.
I/O                     |   pg_statio_all_sequences	    |   One row for each sequence in the current database, showing statistics about I/O on that specific sequence.
I/O                     |   pg_statio_sys_sequences	    |   Same as pg_statio_all_sequences, except that only system sequences are shown. (Presently, no system sequences are defined, so this view is always empty.)
I/O                     |   pg_statio_user_sequences	|   Same as pg_statio_all_sequences, except that only user sequences are shown.

More information in the [docs](https://www.postgresql.org/docs/current/monitoring-stats.html#MONITORING-STATS-VIEWS-TABLE)

## Usage Example

### I/O Overview

In [ ]:
SELECT 
    backend_type,
    CASE WHEN object = 'temp relation' THEN true ELSE false END is_temp,
	context,
	reads,
	read_time,
	writes,
	write_time,
	hits buffer_hits,
	evictions buffer_flushed_for_other_use,
	fsyncs,
	fsync_time,
    stats_reset
FROM pg_stat_io 
WHERE reads <> 0 OR writes <> 0 OR extends <> 0;

### Index VS Sequential Access

In [ ]:
WITH calc AS (
SELECT
	relid AS oid,
	schemaname AS schema,
	relname AS relation,
	idx_tup_fetch,
	seq_tup_read,
	CASE WHEN idx_tup_fetch + seq_tup_read = 0 THEN NULL ELSE idx_tup_fetch + seq_tup_read END AS total_read
FROM pg_stat_all_tables
)

SELECT
	oid,
	schema,
	relation,
	idx_tup_fetch,
	seq_tup_read,
	COALESCE(total_read, 0) AS total_read,
	ROUND(idx_tup_fetch::numeric / total_read, 2) AS index_scan_percentage
FROM calc
ORDER BY index_scan_percentage;